# Všechny možné výrazy
Bez nutnosti cokoliv nastavovat, zapínat a zabezpečovat můžeme pracovat i komplexními hodnotami, a to pomocí příslušných výrazů. Následují ukázky zpracování XML, geografických dat a JSON.

## XML
Následující skript využívá XML reprezentace událostí z Extended Event session zvane **system_health**. Proto je ukázka mírně obohacena o následující prvky:
- funkce **sys.fn_xe_file_target_read_file** - tato funkce umožňuje číst data XEvents uložená v souborech.
- příprava dat z XEvents pomocí konverze na XML a pomocí CTE
- využití katalogu **sys.messages** pro zobrazení obecného textu nalezených chybových hlášení

In [ ]:
;with system_health as
(
select cast(event_data as xml) as event_data 
from sys.fn_xe_file_target_read_file('C:\Program Files\Microsoft SQL Server\MSSQL16.MSSQLSERVER\MSSQL\Log\*.xel', null, null, null)
)
select distinct 
	c.value('(data[@name="error_number"]/value)[1]', 'int') as cislo_chyby
	, c.value('(data[@name="severity"]/value)[1]', 'int') as uroven
	-- , c.value('(data[@name="message"]/value)[1]', 'nvarchar(1000)') as text_chyby
	, m.text
from system_health
	cross apply event_data.nodes('event[@name="error_reported"]') as t(c)
	join sys.messages as m on c.value('(data[@name="error_number"]/value)[1]', 'int') = m.message_id
where m.language_id = 1029

## Geometry, geography
Nakreslíme plošný objekt pomocí OGC výrazu a najdeme na mapě Vivo Hostivař. 

*Pozn.: Všechny ukázky s grafickým výstupem jsou čitelné pouze v SSMS!*

In [ ]:
declare @ctverecek geometry = 'POLYGON((0 0, 0 3, 3 3, 3 0, 0 0))'
select @ctverecek

In [ ]:
select * from sys.spatial_reference_systems where spatial_reference_id = 4326

In [ ]:
declare @kdeJsme geography = geography::STGeomFromText('POINT(50.05339 14.51777)', 4326)
select @kdeJsme
select @kdeJsme.STBuffer(1)

## JSON
JSON nemá na SQL Serveru svůj datový typ, proto se používají funkce nad řetězcem. Ukázka otestuje vstupní řetězec na validitu, a pak z něj získá hodnotu.

In [ ]:
declare @json nvarchar(100) = N'{"jmeno": "Pepa"}'
select ISJSON(@json)

select JSON_QUERY(@json, N'$') as json_piece, JSON_VALUE(@json, N'$.jmeno') as json_attr